
## Часть 2. Использование модели

**Цель:** отработать навыки адаптации готовых моделей для решения прикладной задачи на русском языке, а также создание небольших демо для задач.



### [2 балла] Добавить модель переводчика

У вас уже есть готовая модель, которая может по картинке отвечать на текстовые запросы к картинке. Ваша цель — обобщить эту модель на русский язык, добавив модель переводчик, которая будет переводить запрос на русском языке в запрос на английском языке и передавать его модели. За основу вы можете взять языковую модель (например, https://huggingface.co/Helsinki-NLP/opus-mt-ru-en). Альтернативой может стать реализация функции, делающий api вызов, к приложению переводчика (например, https://libretranslate.com/).

---

**Ожидаемый результат.** В качестве результата в этой секции вам нужно предоставить функции, которые делают перевод с русского на английского и делает инференс модели DocVQA и выводит ответ на русском языке. (В качестве примеров вопросов, можете использовать данные из датасета).


In [14]:
from transformers import MarianMTModel, MarianTokenizer
from transformers import LayoutLMv3ForQuestionAnswering, LayoutLMv3Processor
from datasets import load_dataset
from PIL import Image
import torch

ru_en_tokenizer = MarianTokenizer.from_pretrained('Helsinki-NLP/opus-mt-ru-en')
ru_en_model = MarianMTModel.from_pretrained('Helsinki-NLP/opus-mt-ru-en')
en_ru_tokenizer = MarianTokenizer.from_pretrained('Helsinki-NLP/opus-mt-en-ru')
en_ru_model = MarianMTModel.from_pretrained('Helsinki-NLP/opus-mt-en-ru')

def translate_ru2en(text):
    inputs = ru_en_tokenizer([text], return_tensors="pt", padding=True)
    translated = ru_en_model.generate(**inputs)
    tgt_text = ru_en_tokenizer.batch_decode(translated, skip_special_tokens=True)[0]
    return tgt_text

def translate_en2ru(text):
    inputs = en_ru_tokenizer([text], return_tensors="pt", padding=True)
    translated = en_ru_model.generate(**inputs)
    tgt_text = en_ru_tokenizer.batch_decode(translated, skip_special_tokens=True)[0]
    return tgt_text

model_name = 'microsoft/layoutlmv3-base'
docvqa_model = LayoutLMv3ForQuestionAnswering.from_pretrained(model_name)
docvqa_processor = LayoutLMv3Processor.from_pretrained(model_name)

def docvqa_inference(question_en, model, processor, image):
    encoding = processor(image, question_en, return_tensors="pt")
    outputs = model(**encoding)
    answer_start = torch.argmax(outputs.start_logits)
    answer_end = torch.argmax(outputs.end_logits) + 1
    input_ids = encoding.input_ids[0][answer_start:answer_end]
    answer = processor.tokenizer.decode(input_ids, skip_special_tokens=True)
    return answer

def ru_inference(question_ru, model, processor, image):
    question_en = translate_ru2en(question_ru)
    answer_en = docvqa_inference(question_en, model, processor, image)
    answer_ru = translate_en2ru(answer_en)
    return answer_ru

dataset = load_dataset("nielsr/docvqa_1200_examples", split="test[:1]")
example = dataset[0]

doc_image = example['image'].convert("RGB")
question_ru = "Как называется документ?"

answer = ru_inference(question_ru, docvqa_model, docvqa_processor, doc_image)
print("Вопрос:", question_ru)
print("Ответ:", answer)


/usr/local/lib/python3.12/dist-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")
Some weights of LayoutLMv3ForQuestionAnswering were not initialized from the model checkpoint at microsoft/layoutlmv3-base and are newly initialized: ['qa_outputs.dense.bias', 'qa_outputs.dense.weight', 'qa_outputs.out_proj.bias', 'qa_outputs.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.12/dist-packages/transformers/modeling_utils.py:1621: FutureWarning: The `device` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(


Вопрос: Как называется документ?
Ответ: Я не знаю, что делать.


### [2 балла] Сделать демо на gradio

Модель готова! Теперь было бы круто, если модель можно было захостить и оттестировать на практике. В этом задании вам нужно будет реализовать демо на gradio, которое будет принимать изображение и вопрос, а далее выдавать ответ. Пример демо, аналогично которому вам нужно реализовать модель: https://huggingface.co/spaces/nielsr/comparing-VQA-models.


**Подсказка:** во вкладке `Files` на демо вы можете посмотреть реализацию, там нужно заменить инференс, используемой модели, на инференс нашей модели с переводом


**Ожидаемый результат.** В качестве результата в этой секции вам нужно код для запуска демо на градио и видеозапись его работы, где реализован описанный выше функционал. Видео прикрепляйте отдельным файлом.

In [6]:
import nest_asyncio
nest_asyncio.apply()

import gradio as gr
from transformers import MarianMTModel, MarianTokenizer
from transformers import LayoutLMv3ForQuestionAnswering, LayoutLMv3Processor
import torch

ru_en_tokenizer = MarianTokenizer.from_pretrained('Helsinki-NLP/opus-mt-ru-en')
ru_en_model = MarianMTModel.from_pretrained('Helsinki-NLP/opus-mt-ru-en')
en_ru_tokenizer = MarianTokenizer.from_pretrained('Helsinki-NLP/opus-mt-en-ru')
en_ru_model = MarianMTModel.from_pretrained('Helsinki-NLP/opus-mt-en-ru')

model_name = 'microsoft/layoutlmv3-base'
docvqa_model = LayoutLMv3ForQuestionAnswering.from_pretrained(model_name)
docvqa_processor = LayoutLMv3Processor.from_pretrained(model_name)

device = "cuda" if torch.cuda.is_available() else "cpu"

ru_en_model.to(device)
en_ru_model.to(device)
docvqa_model.to(device)


def translate_ru2en(text):
    inputs = ru_en_tokenizer([text], return_tensors="pt", padding=True, truncation=True, max_length=512)
    inputs = {k: v.to(device) for k, v in inputs.items()}
    translated = ru_en_model.generate(**inputs)
    tgt_text = ru_en_tokenizer.batch_decode(translated, skip_special_tokens=True)[0]
    return tgt_text


def translate_en2ru(text):
    inputs = en_ru_tokenizer([text], return_tensors="pt", padding=True, truncation=True, max_length=512)
    inputs = {k: v.to(device) for k, v in inputs.items()}
    translated = en_ru_model.generate(**inputs)
    tgt_text = en_ru_tokenizer.batch_decode(translated, skip_special_tokens=True)[0]
    return tgt_text


def docvqa_inference(question_en, model, processor, image):
    encoding = processor(image, question_en, return_tensors="pt", padding="max_length", truncation=True, max_length=512)
    encoding = {k: v.to(device) for k, v in encoding.items()}
    outputs = model(**encoding)
    answer_start = torch.argmax(outputs.start_logits)
    answer_end = torch.argmax(outputs.end_logits) + 1
    input_ids = encoding['input_ids'][0][answer_start:answer_end]
    answer = processor.tokenizer.decode(input_ids, skip_special_tokens=True)
    return answer


def generate_answer_docvqa(image, question_ru):
    if image is None or not question_ru.strip():
        return "Пожалуйста, загрузите изображение и введите вопрос."
    question_en = translate_ru2en(question_ru)
    answer_en = docvqa_inference(question_en, docvqa_model, docvqa_processor, image)
    answer_ru = translate_en2ru(answer_en) if answer_en.strip() != "" else "Ответ не найден."
    return answer_ru


iface = gr.Interface(
    fn=generate_answer_docvqa,
    inputs=[gr.Image(type="pil"), gr.Textbox(label="Введите вопрос на русском")],
    outputs=gr.Textbox(label="Ответ"),
    title="DocVQA с переводом",
    description="Загрузите изображение документа и задайте вопрос на русском, получите ответ на русском."
)

/usr/local/lib/python3.12/dist-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")
Some weights of LayoutLMv3ForQuestionAnswering were not initialized from the model checkpoint at microsoft/layoutlmv3-base and are newly initialized: ['qa_outputs.dense.bias', 'qa_outputs.dense.weight', 'qa_outputs.out_proj.bias', 'qa_outputs.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [7]:
iface.launch(debug=True)

It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://42ca6a1322e98229b6.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


ERROR:    Exception in ASGI application
Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/uvicorn/protocols/http/h11_impl.py", line 403, in run_asgi
    result = await app(  # type: ignore[func-returns-value]
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/uvicorn/middleware/proxy_headers.py", line 60, in __call__
    return await self.app(scope, receive, send)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/fastapi/applications.py", line 1134, in __call__
    await super().__call__(scope, receive, send)
  File "/usr/local/lib/python3.12/dist-packages/starlette/applications.py", line 113, in __call__
    await self.middleware_stack(scope, receive, send)
  File "/usr/local/lib/python3.12/dist-packages/starlette/middleware/errors.py", line 186, in __call__
    raise exc
  File "/usr/local/lib/python3.12/dist-packages/starlette/middleware/errors.py",

Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://42ca6a1322e98229b6.gradio.live


### [4 балла] Ответы на вопросы голосом

Демо готово! Но кто хочет писать вопросы текстом?
Здесь вам предстоить улучшить ваше демо, чтобы оно могло принимать вопросы голосом. За основу вам предлагается рассмотреть демо https://www.gradio.app/guides/real-time-speech-recognition и добавить соответствуещее окошко в ваше демо. Также вы можете добавить text-to-speech модель, чтобы оно озвучило текстовый ответ (дополнительный балл к оценке).

---

**Ожидаемый результат.** В качестве результата в этой секции вам нужно код для запуска демо на градио и видеозапись его работы, где реализован описанный выше функционал.

In [ ]:
# TODO Реализация демо на gradio с голосовым вводом